In [5]:
import os
import autokeras as ak
import tensorflow as tf
import kerastuner
import pandas as pd
import numpy as np
import keras.backend as K

def true_positives(y_test, y_pred):
    return K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))

def false_positives(y_test, y_pred):
    return K.sum(K.abs(y_test - 1) * y_pred)

def false_negatives(y_test, y_pred):
    return K.sum(y_test * K.abs(y_pred - 1))

def true_negatives(y_test, y_pred):
    return K.sum((y_test - 1) * (y_pred - 1))

def precision_func(y_test, y_pred):
    true_positives = K.sum(y_test * y_pred)
    false_positives = K.sum(K.abs(y_test - 1) * y_pred)
    return true_positives / (true_positives + false_positives + K.epsilon())

def recall_func(y_test, y_pred):
    true_positives = K.sum(y_test * y_pred)
    false_negatives = K.sum(y_test * K.abs(y_pred - 1))
    return true_positives / (true_positives + false_negatives + K.epsilon())

def f1_1_func(y_test, y_pred):
    precision = precision_func(y_test, y_pred)
    recall = recall_func(y_test, y_pred)
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

def specificity_func(y_test, y_pred):
    true_negatives = K.sum((y_test - 1) * (y_pred - 1))
    false_positives = K.sum(K.abs(y_test - 1) * y_pred)
    return true_negatives / (true_negatives + false_positives + K.epsilon())

def npv_func(y_test, y_pred):
    true_negatives = K.sum((y_test - 1) * (y_pred - 1))
    false_negatives = K.sum(y_test * K.abs(y_pred - 1))
    return true_negatives / (true_negatives + false_negatives + K.epsilon())

def f1_0_func(y_test, y_pred):
    specificity = specificity_func(y_test, y_pred)
    npv = npv_func(y_test, y_pred)
    return 2 * (specificity * npv) / (specificity + npv + K.epsilon())

def f1_macro_func(y_test, y_pred):
    f1_1 = f1_1_func(y_test, y_pred)
    f1_0 = f1_0_func(y_test, y_pred)
    return (f1_1 + f1_0) / 2

# -------------------------------------------------------------------------------------------------

SEGMENTS_LENGTH = 3
EXPERIMENT_ID = F'Base_{SEGMENTS_LENGTH}s'

if not os.path.exists('Tensorboard'): os.makedirs('Tensorboard')
if not os.path.exists('Models'): os.makedirs('Models')
if not os.path.exists('Projects'): os.makedirs('Projects')
tensorboard_dir = f'Tensorboard//{EXPERIMENT_ID}'
if not os.path.exists(f'Tensorboard//{EXPERIMENT_ID}'): os.makedirs(f'Tensorboard//{EXPERIMENT_ID}')


In [7]:
# Data reading

data_dir = 'D:\FINKI\8_dps\Project\DATA\MODELS_DATA\\base_models_all_dft_coefficients'
X_train = pd.read_csv(f'{data_dir}\\segments_{SEGMENTS_LENGTH}s_train.csv')
X_test = pd.read_csv(f'{data_dir}\\segments_{SEGMENTS_LENGTH}s_test.csv')

y_train = X_train.pop('episode')
y_test = X_test.pop('episode')


In [10]:
classifier = ak.StructuredDataClassifier(
    project_name = f"Projects/{EXPERIMENT_ID}",
    objective = kerastuner.Objective('f1_macro_func', direction='max'),
    metrics=[f1_1_func, "accuracy", f1_macro_func, f1_0_func],
    overwrite=True,
    max_trials=2,
)

tensorboard_callback_train = tf.keras.callbacks.TensorBoard(log_dir=f'Tensorboard//{EXPERIMENT_ID}')
Early_Stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

classifier.fit(
    X_train,
    y_train,
    epochs=2,
    verbose=10,
    callbacks=[Early_Stopping, tensorboard_callback_train],
    batch_size=1000,
)

clf_best_model = classifier.export_model()

# %%
try: clf_best_model.save(f"Models/{EXPERIMENT_ID}", save_format="tf")
except: clf_best_model.save(f"Models/{EXPERIMENT_ID}.h5")
# %%
from tensorflow.keras.models import load_model
Custom_Objects = ak.CUSTOM_OBJECTS
Custom_Objects["f1_0_func"] = f1_0_func
Custom_Objects["f1_1_func"] = f1_1_func
Custom_Objects["f1_macro_func"] = f1_macro_func
loaded_model = load_model(f"Models/{EXPERIMENT_ID}", custom_objects=Custom_Objects)


print("\n================================= Best Modal Evaluattion ==============================\n")
print('\n Loss: {accuracy[0]} \n F1 score: {accuracy[1]} \n Accuracy: {accuracy[2]}'.format(
    accuracy=loaded_model.evaluate(np.array(X_test).astype(str), y_test, batch_size=50001))
)
print("\n=======================================================================================\n")

print("+++++++++++++++++++++++++++++++++++++++++++++++++++++ FINISHED SCRIPT +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")


y_pred = loaded_model.predict(np.array(X_test).astype(str)).astype(int).ravel()

pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).to_excel('model_predictions.xlsx')
print(pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}))

Trial 2 Complete [00h 00m 04s]
f1_macro_func: 0.521094024181366

Best f1_macro_func So Far: 0.5269290804862976
Total elapsed time: 00h 00m 07s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
Epoch 2/2
INFO:tensorflow:Assets written to: .\Projects/Base_3s\best_model\assets


ValueError: Unable to restore custom object of type _tf_keras_metric currently. Please make sure that the layer implements `get_config`and `from_config` when saving. In addition, please use the `custom_objects` arg when calling `load_model()`.